In [ ]:
# 나눔고딕 받아오기 - 런타임 초기화 후 다시 시작!!!
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 0s (41.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV,  cross_val_score
from sklearn import preprocessing 
from sklearn import linear_model as lm
from sklearn.preprocessing import  LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler 
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import Ridge, Lasso, LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn import tree
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
import xgboost as xgb
from scipy.stats import uniform, randint

In [ ]:
import lightgbm as lgb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def seed_everything(seed: int = 0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/딥인치즈/Data/f5.csv")
data.head()

,month,day,season,time,rainfall,dust,dong,wind,humidity,snow,...,weekday,heat,station_num,station_name,station_place,place,latitude,longitude,installation_date,target
0,1,1,winter,0,0.0,23.0,가락동,1.9,41.0,0.0,...,5.0,-4.76,4453,위례중앙푸르지오 201동 앞,위례중앙푸르지오 201동 앞,위례광장로 290,37.474346,127.143265,2020-12-30,1
1,1,1,winter,0,0.0,23.0,거여동,1.9,41.0,0.0,...,5.0,-4.76,1227,거여역 8번출구 뒤,거여역 8번출구 뒤,서울특별시 송파구 오금로 지하 499,37.493126,127.143822,2017-06-27,1
2,1,1,winter,0,0.0,23.0,거여동,1.9,41.0,0.0,...,5.0,-4.76,4895,위례포레샤인15단지 상가,위례포레샤인15단지 상가,송파구 거여동 604,37.486416,127.142059,2021-11-10,1
3,1,1,winter,0,0.0,23.0,마천동,1.9,41.0,0.0,...,5.0,-4.76,2642,마천동 성내천 쉼터,마천동 성내천 쉼터,성내천로 179,37.501495,127.147781,2020-01-15,1
4,1,1,winter,0,0.0,23.0,문정동,1.9,41.0,0.0,...,5.0,-4.76,1244,문정 법조단지8,문정 법조단지8,서울특별시 송파구 법원로 101,37.481537,127.122124,2017-06-27,1


In [ ]:
data[['target']].value_counts()

target
1         331621
2         184397
3         107878
4          67154
5          41148
6          26682
7          16639
8          11574
9           7170
10          5075
11          3102
12          2421
13          1517
14          1272
15           763
16           725
17           402
18           341
20           221
19           217
22           154
21           126
23            88
24            73
25            62
26            60
28            26
27            25
29            24
30            18
32            13
31             9
33             6
34             5
36             3
40             2
35             1
41             1
42             1
48             1
56             1
dtype: int64

# 잠실동, 방이동만

In [ ]:
jam=data[data['dong']=='잠실동']
jam.drop(['station_num','station_name','place'],axis=1,inplace=True)
jam.drop(['latitude','longitude'],axis=1,inplace=True)

In [ ]:
bang=data[data['dong']=='방이동']
bang.drop(['station_num','station_name','place'],axis=1,inplace=True)
bang.drop(['latitude','longitude'],axis=1,inplace=True)

## 잠실동

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
import pandas as pd
A=jam[['season']]
for i in A:
  jam[i]=encoder.fit_transform(jam[i])

In [ ]:
jam['year']=2021
jam['year'].loc[jam['month']<=4]=2022

jam.reset_index(inplace=True)
jam.drop(['index','dong'],axis=1,inplace=True)

jam['installation_year']=jam['installation_date'].apply(lambda x:str(x).split('-')[0])
jam.drop(['installation_date'],axis=1,inplace=True)

jam['installation_period']=0
for i in range(len(jam)):
  jam['installation_period'].iloc[i]=int(jam['year'].iloc[i])-int(jam['installation_year'].iloc[i])
jam.drop([ 'installation_year','year'], axis=1, inplace=True)

In [ ]:
jam.head()

,month,day,season,time,rainfall,dust,wind,humidity,snow,temp,holiday,weekday,heat,station_place,target,installation_period
0,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,종합운동장역 4번출구,3,5
1,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,잠실역 6번출구,2,5
2,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,잠실트리지움310동 옆,1,5
3,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,석촌호수교차로(동호 팔각정 앞),2,4
4,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,삼전역 1번출구,2,4


In [ ]:
numerical=list(jam.columns)
categorical=['month','day','time','season','holiday','station_place','weekday', 'target', 'installation_period']

for i in categorical:
  numerical.remove(i)

In [ ]:
jam.to_csv("/content/drive/MyDrive/딥인치즈/Data/하경_기온 데이터/jamsil_1.csv", index=False)

## 방이동

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
import pandas as pd
A=bang[['season']]
for i in A:
  bang[i]=encoder.fit_transform(bang[i])

In [ ]:
bang['year']=2021
bang['year'].loc[bang['month']<=4]=2022

bang.reset_index(inplace=True)
bang.drop(['index','dong'],axis=1,inplace=True)

bang['installation_year']=bang['installation_date'].apply(lambda x:str(x).split('-')[0])
bang.drop(['installation_date'],axis=1,inplace=True)

bang['installation_period']=0
for i in range(len(bang)):
  bang['installation_period'].iloc[i]=int(bang['year'].iloc[i])-int(bang['installation_year'].iloc[i])
bang.drop([ 'installation_year','year'], axis=1, inplace=True)

In [ ]:
bang.head()

,month,day,season,time,rainfall,dust,wind,humidity,snow,temp,holiday,weekday,heat,station_place,target,installation_period
0,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,26,3,5
1,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,22,2,5
2,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,25,1,5
3,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,7,2,4
4,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,3,2,4


In [ ]:
numerical=list(bang.columns)
categorical=['month','day','time','season','holiday','station_place','weekday', 'target', 'installation_period']

for i in categorical:
  numerical.remove(i)

In [ ]:
jam.to_csv("/content/drive/MyDrive/딥인치즈/Data/하경_기온 데이터/bang_1.csv", index=False)

## 전체 정류소 대상 베이스라인(잠실동)

In [ ]:
jam=pd.read_csv("/content/drive/MyDrive/딥인치즈/Data/하경_기온 데이터/jamsil_1.csv")
jam.head()

,month,day,season,time,rainfall,dust,wind,humidity,snow,temp,holiday,weekday,heat,station_place,target,installation_period
0,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,종합운동장역 4번출구,3,5
1,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,잠실역 6번출구,2,5
2,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,잠실트리지움310동 옆,1,5
3,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,석촌호수교차로(동호 팔각정 앞),2,4
4,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,삼전역 1번출구,2,4


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
import pandas as pd
A=jam[['station_place']]
for i in A:
  jam[i]=encoder.fit_transform(jam[i])

In [ ]:
jam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130263 entries, 0 to 130262
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                130263 non-null  int64  
 1   day                  130263 non-null  int64  
 2   season               130263 non-null  int64  
 3   time                 130263 non-null  int64  
 4   rainfall             130263 non-null  float64
 5   dust                 130263 non-null  float64
 6   wind                 130263 non-null  float64
 7   humidity             130263 non-null  float64
 8   snow                 130263 non-null  float64
 9   temp                 130263 non-null  float64
 10  holiday              130263 non-null  int64  
 11  weekday              130263 non-null  float64
 12  heat                 130263 non-null  float64
 13  station_place        130263 non-null  int64  
 14  target               130263 non-null  int64  
 15  installation_peri

In [ ]:
X = jam.drop(['target'], axis = 1)
y = jam['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_log_error
def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

In [ ]:
X_train.head()

,month,day,season,time,rainfall,dust,wind,humidity,snow,temp,holiday,weekday,heat,station_place,installation_period
85804,9,1,0,15,0.0,14.0,4.2,75.0,0.0,23.4,0,2.0,21.13,16,1
55743,6,13,2,14,0.0,30.0,2.6,56.0,0.0,29.1,1,6.0,24.92,20,1
116025,11,17,0,13,0.0,38.0,2.1,44.0,0.0,12.0,0,2.0,7.40,17,1
111709,11,5,0,21,0.0,92.0,0.9,60.0,0.0,13.4,0,4.0,12.93,5,3
26063,3,24,1,8,0.0,28.0,1.7,75.0,0.0,5.5,0,3.0,7.90,15,4


파라미터 없이 기본 회귀 모델로만 성능 확인하기

In [ ]:
modellist=[LinearRegression(), lm.Ridge(),lm.Lasso(), DecisionTreeRegressor(), RandomForestRegressor(),xgb.XGBRegressor(objective='reg:squarederror'),ExtraTreesRegressor(),lgb.LGBMRegressor()]

for model in modellist:
  model.fit(X_train,y_train)
  y_pred=model.predict(X_test)
  print()
  print("Model R-Square : " ,r2_score(y_test,y_pred))
  print("Model MSE : " ,mean_squared_error(y_test,y_pred))
  print("Model RMSE : " ,np.sqrt(mean_squared_error(y_test,y_pred)))
  print("Model RMSLE : " ,rmsle(y_test,y_pred))

LinearRegression()


Model R-Square :  0.15957127179702202
Model MSE :  4.871162360079691
Model RMSE :  2.2070709911735262
Model RMSLE :  0.48356289897156796


Ridge()


Model R-Square :  0.1595712947691985
Model MSE :  4.871162226931933
Model RMSE :  2.207070961009621
Model RMSLE :  0.4835621820763625


Lasso()

...


Model R-Square :  0.36361071717064497
Model MSE :  3.688540642232509
Model RMSE :  1.9205573780110057
Model RMSLE :  0.4138909955933791


ExtraTreesRegressor()


Model R-Square :  0.48835220097031484
Model MSE :  2.965533443365447
Model RMSE :  1.7220724268640524
Model RMSLE :  0.39000734480143834


LGBMRegressor()


Model R-Square :  0.53665217756629
Model MSE :  2.6855846266583856
Model RMSE :  1.6387753435594477
Model RMSLE :  0.3688119162675551


## heatmap

In [ ]:
jam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130263 entries, 0 to 130262
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                130263 non-null  int64  
 1   day                  130263 non-null  int64  
 2   season               130263 non-null  int64  
 3   time                 130263 non-null  int64  
 4   rainfall             130263 non-null  float64
 5   dust                 130263 non-null  float64
 6   wind                 130263 non-null  float64
 7   humidity             130263 non-null  float64
 8   snow                 130263 non-null  float64
 9   temp                 130263 non-null  float64
 10  holiday              130263 non-null  int64  
 11  weekday              130263 non-null  float64
 12  heat                 130263 non-null  float64
 13  station_place        130263 non-null  int64  
 14  target               130263 non-null  int64  
 15  installation_peri

In [ ]:
x=jam.columns
jam.corr().loc[x, x].style.background_gradient().set_precision(2).set_properties(**{'font-size': '11pt'})

,month,day,season,time,rainfall,dust,wind,humidity,snow,temp,holiday,weekday,heat,station_place,target,installation_period
month,1.00,-0.02,-0.39,-0.01,0.01,-0.17,-0.12,0.26,-0.04,0.24,-0.03,0.00,0.26,-0.01,0.04,-0.24
day,-0.02,1.00,-0.01,0.01,0.01,-0.03,-0.05,0.01,-0.00,-0.01,0.01,-0.01,-0.00,0.00,-0.01,0.00
season,-0.39,-0.01,1.00,0.00,-0.01,0.06,0.03,-0.11,0.11,-0.31,-0.02,-0.01,-0.29,0.01,-0.16,0.08
time,-0.01,0.01,0.00,1.00,-0.01,0.06,0.16,-0.23,-0.01,0.09,0.01,-0.00,-0.02,0.01,0.28,-0.02
rainfall,0.01,0.01,-0.01,-0.01,1.00,-0.02,0.01,0.11,-0.00,0.02,0.01,-0.00,0.03,0.00,-0.04,0.00
dust,-0.17,-0.03,0.06,0.06,-0.02,1.00,0.03,-0.02,0.00,-0.12,0.01,0.05,-0.14,0.00,0.01,0.05
wind,-0.12,-0.05,0.03,0.16,0.01,0.03,1.00,-0.33,0.00,-0.03,0.05,0.03,-0.11,0.01,0.08,0.03
humidity,0.26,0.01,-0.11,-0.23,0.11,-0.02,-0.33,1.00,0.04,0.16,-0.04,-0.07,0.32,-0.02,-0.12,-0.09
snow,-0.04,-0.00,0.11,-0.01,-0.00,0.00,0.00,0.04,1.00,-0.13,0.03,-0.01,-0.13,0.00,-0.05,0.02
temp,0.24,-0.01,-0.31,0.09,0.02,-0.12,-0.03,0.16,-0.13,1.00,0.02,0.03,0.96,-0.02,0.24,-0.15


## 전체 정류소 대상 베이스라인(방이동)

In [ ]:
bang=pd.read_csv("/content/drive/MyDrive/딥인치즈/Data/하경_기온 데이터/bang_1.csv")
bang.head()

,month,day,season,time,rainfall,dust,wind,humidity,snow,temp,holiday,weekday,heat,station_place,target,installation_period
0,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,26,3,5
1,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,22,2,5
2,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,25,1,5
3,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,7,2,4
4,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,3,2,4


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
import pandas as pd
A=bang[['station_place']]
for i in A:
  bang[i]=encoder.fit_transform(bang[i])

In [ ]:
bang.head()

,month,day,season,time,rainfall,dust,wind,humidity,snow,temp,holiday,weekday,heat,station_place,target,installation_period
0,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,26,3,5
1,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,22,2,5
2,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,25,1,5
3,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,7,2,4
4,1,1,3,0,0.0,23.0,1.9,41.0,0.0,-8.5,1,5.0,-4.76,3,2,4


In [ ]:
X = bang.drop(['target'], axis = 1)
y = bang['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

파라미터 없이 기본 회귀 모델로만 성능 확인하기

In [ ]:
modellist=[LinearRegression(), lm.Ridge(),lm.Lasso(), DecisionTreeRegressor(), RandomForestRegressor(),xgb.XGBRegressor(objective='reg:squarederror'),ExtraTreesRegressor(),lgb.LGBMRegressor()]

for model in modellist:
  model.fit(X_train,y_train)
  y_pred=model.predict(X_test)
  print()
  print("Model R-Square : " ,r2_score(y_test,y_pred))
  print("Model MSE : " ,mean_squared_error(y_test,y_pred))
  print("Model RMSE : " ,np.sqrt(mean_squared_error(y_test,y_pred)))
  print("Model RMSLE : " ,rmsle(y_test,y_pred))

LinearRegression()


Model R-Square :  0.15957127179702202
Model MSE :  4.871162360079691
Model RMSE :  2.2070709911735262
Model RMSLE :  0.48356289897156796


Ridge()


Model R-Square :  0.1595712947691985
Model MSE :  4.871162226931933
Model RMSE :  2.207070961009621
Model RMSLE :  0.4835621820763625


Lasso()

...


Model R-Square :  0.36361071717064497
Model MSE :  3.688540642232509
Model RMSE :  1.9205573780110057
Model RMSLE :  0.4138909955933791


ExtraTreesRegressor()


Model R-Square :  0.48461861313787946
Model MSE :  2.9871734848194063
Model RMSE :  1.7283441453655595
Model RMSLE :  0.3913223334723998


LGBMRegressor()


Model R-Square :  0.53665217756629
Model MSE :  2.6855846266583856
Model RMSE :  1.6387753435594477
Model RMSLE :  0.3688119162675551
